# Word2Vec 3D Embeddings for LSTM

Энэ notebook нь текстийн өгөгдлийг word-level дээр боловсруулж, 3D tensor (samples, max_length, embedding_dim) үүсгэх замаар LSTM-д тохирсон форматтай болгоно.

**Алхамууд:**
1. Текст цэвэрлэх
2. Word tokenization
3. Word2Vec модел сургах
4. 3D embedding tensor үүсгэх
5. Хадгалах

In [ ]:
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import os
from tqdm import tqdm

print("Сангууд ачааллаа")

## 1. Өгөгдөл ачаалах

In [ ]:
# Өгөгдөл ачаалах
df = pd.read_csv('/home/tr1bo/Documents/4. 3A/Эх хэлний боловсруулалт/biydaalt1/data/cleaned_label.csv')

print(f"Нийт мөрийн тоо: {len(df)}")
print(f"Багана: {df.columns.tolist()}")
print("\nЭхний хэдэн мөр:")
df.head()

## 2. Текст цэвэрлэх ба Tokenization

In [ ]:
def clean_text(text):
    """Текст цэвэрлэх функц"""
    # Жижиг үсэг болгох
    text = text.lower()
    # HTML тэмдэгт устгах
    text = re.sub(r'<[^>]+>', '', text)
    # Зөвхөн үсэг, тоо, зай үлдээх
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Олон зай нэг зай болгох
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Текст цэвэрлэх
print("Текст цэвэрлэж байна...")
df['cleaned_text'] = df['review_text'].apply(clean_text)

# Хоосон мөр устгах
df = df[df['cleaned_text'].str.len() > 0].reset_index(drop=True)

print(f"Цэвэрлэсний дараах мөрийн тоо: {len(df)}")
print("\nЖишээ:")
print(f"Анхны: {df['review_text'].iloc[0][:100]}...")
print(f"Цэвэр: {df['cleaned_text'].iloc[0][:100]}...")

In [ ]:
# Үгэнд хувааж list болгох
print("Tokenization хийж байна...")
tokenized_texts = [text.split() for text in tqdm(df['cleaned_text'])]

# Статистик
lengths = [len(tokens) for tokens in tokenized_texts]
print(f"\nТекст бүрийн дундаж үг: {np.mean(lengths):.2f}")
print(f"Хамгийн богино: {np.min(lengths)} үг")
print(f"Хамгийн урт: {np.max(lengths)} үг")
print(f"Median: {np.median(lengths):.2f} үг")

## 3. Word2Vec модел сургах

In [ ]:
# Word2Vec параметрүүд
EMBEDDING_DIM = 100  # Embedding хэмжээ
WINDOW_SIZE = 5      # Context цонхны хэмжээ
MIN_COUNT = 2        # Хамгийн багадаа 2 удаа давтагдах үг

print("Word2Vec модел сургаж байна...")
w2v_model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=EMBEDDING_DIM,
    window=WINDOW_SIZE,
    min_count=MIN_COUNT,
    workers=4,
    sg=0,  # CBOW
    epochs=10
)

print(f"\nYocabulary хэмжээ: {len(w2v_model.wv)}")
print(f"Embedding dimension: {EMBEDDING_DIM}")

# Модел хадгалах
model_dir = '/home/tr1bo/Documents/4. 3A/Эх хэлний боловсруулалт/biydaalt1/models/word2vec'
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, 'word2vec_3d.model')
w2v_model.save(model_path)
print(f"\nWord2Vec модел хадгалагдсан: {model_path}")

## 4. 3D Embedding Tensor үүсгэх

LSTM-д оролт өгөхийн тулд өгөгдлийг **(samples, max_length, embedding_dim)** хэлбэрт оруулах шаардлагатай.

In [ ]:
# Maximum sequence length тодорхойлох
MAX_LENGTH = 200  # Ихэнх текстийг хамрах утга

print(f"Maximum sequence length: {MAX_LENGTH}")
print(f"Энэ нь текстийн {np.percentile(lengths, 95):.0f} percentile-ээс их байна")

# OOV (Out of Vocabulary) үгэнд зориулсан нөөц вектор
oov_vector = np.zeros(EMBEDDING_DIM)

def text_to_embedding_sequence(tokens, max_length):
    """
    Tokenized текстийг embedding sequence болгох
    
    Args:
        tokens: List of words
        max_length: Maximum sequence length
        
    Returns:
        2D numpy array (max_length, embedding_dim)
    """
    sequence = []
    
    for word in tokens[:max_length]:  # Maximum length хүртэл авах
        if word in w2v_model.wv:
            sequence.append(w2v_model.wv[word])
        else:
            sequence.append(oov_vector)  # OOV үгэд нөөц вектор
    
    # Padding: богино текстийг нөхөх
    while len(sequence) < max_length:
        sequence.append(oov_vector)
    
    return np.array(sequence)

# 3D tensor үүсгэх
print("\n3D embedding tensor үүсгэж байна...")
embeddings_3d = []

for tokens in tqdm(tokenized_texts):
    emb_seq = text_to_embedding_sequence(tokens, MAX_LENGTH)
    embeddings_3d.append(emb_seq)

embeddings_3d = np.array(embeddings_3d)

print(f"\n3D Embedding хэмжээ: {embeddings_3d.shape}")
print(f"  - Samples: {embeddings_3d.shape[0]}")
print(f"  - Max Length (timesteps): {embeddings_3d.shape[1]}")
print(f"  - Embedding Dimension: {embeddings_3d.shape[2]}")

## 5. Labels бэлтгэх

In [ ]:
# Labels ачаалах
labels = df['sentiment_label'].values

print(f"Labels хэмжээ: {labels.shape}")
print(f"Positive: {np.sum(labels == 1)}")
print(f"Negative: {np.sum(labels == 0)}")

## 6. Хадгалах

In [ ]:
# 3D embeddings хадгалах
embeddings_path = os.path.join(model_dir, 'word2vec_3d_embeddings.npy')
np.save(embeddings_path, embeddings_3d)
print(f"3D Embeddings хадгалагдсан: {embeddings_path}")

# Labels хадгалах
labels_path = os.path.join(model_dir, 'word2vec_3d_labels.npy')
np.save(labels_path, labels)
print(f"Labels хадгалагдсан: {labels_path}")

# Metadata хадгалах
metadata = {
    'max_length': MAX_LENGTH,
    'embedding_dim': EMBEDDING_DIM,
    'vocab_size': len(w2v_model.wv),
    'num_samples': len(embeddings_3d),
    'shape': embeddings_3d.shape
}

metadata_path = os.path.join(model_dir, 'word2vec_3d_metadata.pkl')
with open(metadata_path, 'wb') as f:
    pickle.dump(metadata, f)
print(f"Metadata хадгалагдсан: {metadata_path}")

print("\n✅ Бүх файлууд амжилттай хадгалагдлаа!")

## 7. Хадгалсан файлуудыг шалгах

In [ ]:
# Файлуудыг дахин ачаалж шалгах
print("Файлуудыг дахин ачаалж байна...")

# Embeddings ачаалах
X_loaded = np.load(embeddings_path)
print(f"✓ 3D Embeddings: {X_loaded.shape}")

# Labels ачаалах
y_loaded = np.load(labels_path)
print(f"✓ Labels: {y_loaded.shape}")

# Metadata ачаалах
with open(metadata_path, 'rb') as f:
    meta_loaded = pickle.load(f)
print(f"✓ Metadata:")
for key, value in meta_loaded.items():
    print(f"  - {key}: {value}")

print("\n✅ Бүх файлууд зөв ачааллагдлаа!")

## 8. LSTM-д ашиглах жишээ

Үүсгэсэн 3D embedding-ийг LSTM модел дээр ашиглах жишээ:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split

# Train/test хуваах
X_train, X_test, y_train, y_test = train_test_split(
    X_loaded, y_loaded, test_size=0.2, random_state=42, stratify=y_loaded
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# LSTM модел үүсгэх
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=False), input_shape=(MAX_LENGTH, EMBEDDING_DIM)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\nМоделийн архитектур:")
model.summary()

print("\n✅ LSTM модел бэлэн! model.fit() дуудаж сургаж болно.")